ANÁLISIS DE SENTIMIENTO Y CATEGORIZACIÓN DE TEXTO (NO ENTRENADO)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
import torch
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Descargar recursos de NLTK 
nltk.download('punkt')
nltk.download('wordnet')

# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Cargar el modelo BERT
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Leer el archivo Excel
file_path = 'C:/Users/joey_/Desktop/DB (RESPALDOS)/Comments - copia.xlsx'
df = pd.read_excel(file_path)

# 1. Análisis de sentimiento
def analyze_sentiment(comment):
    comment = str(comment).strip()
    if not comment or comment == "nan":
        return "NEUTRAL", 0.0
    result = sentiment_analyzer(comment, truncation=True, max_length=512)[0]
    label = result['label']
    score = result['score']
    if label == "POSITIVE":
        polarity = score
        sentiment = "Positive" if score > 0.7 else "Neutral"
    else:
        polarity = -score
        sentiment = "Negative" if score > 0.7 else "Neutral"
    return sentiment, polarity

df[['sentiment', 'polarity']] = df["TRANSLATED COMMENTS"].apply(analyze_sentiment).apply(pd.Series)

print("Distribución de sentimientos:")
print(df['sentiment'].value_counts())

df_nn = df[df['sentiment'].isin(['Negative', 'Neutral'])].copy()
comments_nn = df_nn["TRANSLATED COMMENTS"].tolist()
print(f"\nNúmero de comentarios Negativos y Neutrales: {len(comments_nn)}")

print("\nEjemplos de comentarios con puntajes:")
print(df[["TRANSLATED COMMENTS", 'sentiment', 'polarity']].head(10))

# 2. Listado predefinido de issues (en forma base)
predefined_issues = {
    "Slow Response/Delay Issues": ["faster","feedback","slow", "answer", "solution" , "wait", "long", "wait", "late", "take", "forever", "reply", "frustrate"],
    "Poor Agent Knowledge": ["know", "prepare", "confuse", "wrong", "incompetent", "expert"],
    "Poor Support Quality": ["rude", "help", "attitude", "confuse", "incompetent", "understand", "agent"],
    "Fraud/Trust Issues": ["fraud", "scam", "trust", "secure", "money", "pay", "fake", "hack", "safe"],
    "Product and Services": ["account", "service", "term", "policy", "product", "avail", "refund"],
    "Platform Functionality": ["app","api","platform", "system", "error", "bug", "crash", "technical", "happen"],
    "Lack of Communication Channels": ["phone", "email", "contact", "call", "chat", "verbal","operator","agent"],
    "Lack of Response": ["assistance","receive","any","request", "support" ,"contact","respond", "answer", "no one", "get", "feedback", "question", "nobody", "silent","call"],
    "Ineffective Resolution": ["unsuccessful","request","resolve", "problem", "solve", "fix", "nothing", "resolution"],
    "Language Barriers": ["portuguese", "help", "speaker", "native", "spanish", "english", "language", "italian", "translate", "speak"]
}


# Función para lematizar un texto
def lemmatize_text(text):
    words = word_tokenize(text.lower())
    lemmatized_words = {lemmatizer.lemmatize(word) for word in words}
    return lemmatized_words

# Función para asignar issue a un comentario individual
def assign_issue(comment):
    comment_words = lemmatize_text(comment)
    max_overlap = 0
    best_issue = "Other Issues"
    for issue, issue_keywords in predefined_issues.items():
        overlap = len(comment_words.intersection(set(issue_keywords)))
        if overlap > max_overlap or (overlap == max_overlap and issue in ["Lack of Response", "Language Barriers", "Platform Functionality", "Poor Support Quality", "Poor Agent Knowledge", "Product and Services", "Fraud/Trust Issues", "Lack of Communication Channels"]):
            max_overlap = overlap
            best_issue = issue
    return best_issue if max_overlap > 0 else "Other Issues"

# Asignar issues directamente a cada comentario
df_nn['issue'] = df_nn["TRANSLATED COMMENTS"].apply(assign_issue)

# 3. Validación
print("\nComentarios con sus issues asignados:")
print(df_nn[["TRANSLATED COMMENTS", 'sentiment', 'polarity', 'issue']])

# 4. Visualización de frecuencias
plt.figure(figsize=(12, 6))
issue_counts = df_nn['issue'].value_counts()
sns.barplot(x=issue_counts.values, y=issue_counts.index, palette = "viridis",hue=issue_counts.index)
plt.xlabel('Frecuencia', size=14)
plt.ylabel('Topics/Issues', size=14)
plt.title('Frequency of Topics related to SurveyComments (Negatives Comments)', size=16)
for i, v in enumerate(issue_counts.values):
    plt.text(v, i, str(v), va='center',size=14)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.tight_layout()
plt.show()

c:\Users\joey_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
Traceback (most recent call last):
  File "c:\Users\joey_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Error en una rutina de inicialización de biblioteca de vínculos dinámicos (DLL).


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
df_nn.head(25)

,TRANSLATED COMMENTS,sentiment,polarity,issue
2,...I didn't even get a response!,Negative,-0.999758,Lack of Response
3,2 weeks and no resolution yet,Negative,-0.998362,Ineffective Resolution
7,above-average and customer-unfriendly waiting ...,Negative,-0.999778,Lack of Response
10,After a few days I found out on my own how to ...,Negative,-0.999192,Ineffective Resolution
20,"Answers take far too long, even for urgent pro...",Negative,-0.999647,Slow Response/Delay Issues
21,Apart from receiving emails saying we are proc...,Negative,-0.999522,Lack of Response
23,assistance on missed collections.,Negative,-0.985095,Lack of Response
24,At first it seemed like he didn't understand w...,Negative,-0.996610,Lack of Communication Channels
25,bad,Negative,-0.999782,Other Issues
26,Basically a nice chat but the answer was compl...,Negative,-0.997492,Fraud/Trust Issues


SOLO NEGATIVOS

In [ ]:
df_nn.to_excel('C:/Users/joey_/Desktop/DESK 2025/DB (RESPALDOS)/Sentiments and Topics.xlsx', index=False)